<font size = "5"> **EELS_Tools:  [Spectroscopy](../4_EELS_Tools.ipynb)** </font>

<hr style="height:1px;border-top:4px solid #FF8200" />

# Analysis of Core-Loss EELS Spectra
<br>

[<img src=https://www.coeuscreativegroup.com/wp-content/uploads/2020/04/download-button.png, width=125>](https://raw.githubusercontent.com/pycroscopy/pyTEMlib/main/notebooks/EELS/Analysis_Core_Loss.ipynb) 

[![OpenInColab](https://colab.research.google.com/assets/colab-badge.svg)](
    https://colab.research.google.com/github/pycroscopy/pyTEMlib/blob/main/notebooks/EELS/Analysis_Core_Loss.ipynb)
    
part of 

<font size = "5"> **[pyTEMlib](https://pycroscopy.github.io/pyTEMlib/about.html)**</font>

a [pycroscopy](https://pycroscopy.github.io/pycroscopy/about.html) ecosystem package


Notebook by Gerd Duscher, 2023

Microscopy Facilities<br>
Institute of Advanced Materials & Manufacturing<br>
The University of Tennessee, Knoxville

Model based analysis and quantification of data acquired with transmission electron microscopes




## Content

Quantitative determination of chemical composition from a core-loss EELS spectrum

Relavant notebooks in course [MSE 672: Introduction to TEM]()  give an in depth explanation of the method.

Please cite:

[M. Tian et  al. *Measuring the areal density of nanomaterials by electron energy-loss spectroscopy*
Ultramicroscopy Volume 196, 2019, pages 154-160](https://doi.org/10.1016/j.ultramic.2018.10.009)

as a reference of this quantification method.

## Prerequesites
This notebook will only run with  ``pyTEMlib version 0.2023.5.0`` or higher.

### Install missing packages
This can take a while for the first time. This code will have to run at least once on your computer and for each colab session.

In [1]:
import sys

from pkg_resources import get_distribution, DistributionNotFound

def test_package(package_name):
    """Test if package exists and returns version or -1"""
    try:
        version = (get_distribution(package_name).version)
    except (DistributionNotFound, ImportError) as err:
        version = '-1'
    return version


# pyTEMlib setup ------------------
if test_package('pyTEMlib') < '0.2023.5.0':
    print('installing pyTEMlib')
    !{sys.executable} -m pip install git+https://github.com/pycroscopy/sidpy.git@rama_dev
    !{sys.executable} -m pip install git+https://github.com/pycroscopy/SciFiReaders.git@main -q
    !{sys.executable} -m pip install git+https://github.com/pycroscopy/pyTEMlib.git@main -q --upgrade
    
# ------------------------------
print('done')

done


### Load the relevant packages

In [2]:
%matplotlib ipympl
import numpy as np
import matplotlib.pylab as plt 
import sys

if 'google.colab' in sys.modules:
    from google.colab import output
    output.enable_custom_widget_manager()
    from google.colab import drive
else:
    %gui qt5

import pyTEMlib
import pyTEMlib.file_tools as ft     # File input/ output library
import pyTEMlib.image_tools as it
import pyTEMlib.eels_tools  as eels        # EELS methods 

import pyTEMlib.interactive_eels as ieels  # Dialogs for EELS input and quantification

# For archiving reasons it is a good idea to print the version numbers out at this point
print('pyTEMlib version: ', pyTEMlib.__version__)

__notebook__ = 'analyze_core_loss'
__notebook_version__ = '2023_05_21'

if 'google.colab' in sys.modules:
    drive.mount("/content/drive")

You don't have igor installed.     If you wish to open igor files, you will need to install it     (pip install igor) before attempting.
You don't have gwyfile installed.     If you wish to open .gwy files, you will need to      install it (pip install gwyfile) before attempting.
Symmetry functions of spglib enabled
SimpleITK not installed; Registration Functions for Image Stacks not available
install with: conda install -c simpleitk simpleitk 
pyTEMlib version:  0.2023.5.1


## Load and plot a spectrum

As an example we load the spectrum **1EELS Acquire (high-loss).dm3** from the *example data* folder.

Please see [Loading an EELS Spectrum](LoadEELS.ipynb) for details on storage and plotting.

First a dialog to select a file will apear.

Then the spectrum plot and ``Spectrum Info`` dialog will appear, in which we set the experimental parameters.

Please use the ``Set Energy Scale`` button to change the energy scale. When pressed a new dialog and a cursor will appear in which one is able to set the energy scale based on known features in the spectrum.


In [141]:
# -----Input -------#
load_example = False

if load_example:
    datasets = ft.open_file('../../example_data/EELS_STO3.hf5')
else:  
    fileWidget = ft.FileWidget()

In [143]:
if not load_example:
    datasets = fileWidget.datasets
infoWidget= ieels.InfoWidget(datasets)


AppLayout(children=(GridspecLayout(children=(Dropdown(description='Main Dataset:', layout=Layout(grid_area='wi…

## Chemical Composition 
The fit of the cross-section and background to the spectrum results in the chemical composition. If the calibration is correct this composition is given as areal density in atoms/nm$^2$




In [ ]:
print('Auto Quantification')
eels.auto_chemical_composition(infoWidget.dataset)

###  Fit of Data
A dialog window will open, enter the elements first (0 will open a periodic table) and press 
``Fit Composition`` button (bottom right). Adjust parameters as needed and check fit by pressing the ``Fit Composition`` button again.

Select the ``Region`` checkbox to see which parts of the spectrum you choose to fit.

Changing the multiplier value will make a simulation of your spectrum.

The ``InfoDialog``, if open, still works to change experimental parameters and the energy scale.

In [124]:
compositionWidget = ieels.CompositionWidget(datasets)

AppLayout(children=(GridspecLayout(children=(ToggleButton(value=False, button_style='info', description='Fit A…

### Output of Results

In [97]:
edges = compositionWidget.dataset.metadata['edges']
element = []
areal_density = []
for key, edge in edges.items():
    if key.isdigit():
        element.append(edge['element'])
        areal_density.append(edge['areal_density'])

print('Relative chemical composition of ', compositionWidget.dataset.title)
for i in range(len(element)):
    print(f'{element[i]}: {areal_density[i]/np.sum(areal_density)*100:.1f} %')
    
saved_edges_metadata = edges

Relative chemical composition of  Merve SrTiO4 EELS low loss spectra 0.1 disp 500 ev
Ti: 17.9 %
O: 82.1 %


### Log Data
All the data associated with quantification are stored in the ``metadata`` attribute. The relevant keys of that dictionary are:  ``experiment`` and  ``edges``.

In [7]:
compositionWidget.dataset.metadata.keys()

dict_keys(['experiment', 'filename', 'edges'])

In [8]:
compositionWidget.dataset.view_metadata()

experiment :
	single_exposure_time : 1.0
	exposure_time : 10.0
	number_of_frames : 10
	collection_angle : 40.0
	convergence_angle : 10.0
	microscope : 
	acceleration_voltage : 200000.0
	flux_ppm : 122228.49553999992
	count_conversion : 1
	beam_current : 0
filename : C:\Users\gduscher\Documents\Github\2023-04018-T1-3\Gellerup SrTiO4 EELS Ti and O edge spectra 60mu 1s 0.2 disp.dm3
edges :
	model :
		background : energy_loss:  energy-loss (energy_loss) of size (2048,)
		background-poly_0 : 630802.0962696614
		background-poly_1 : -2915.5152880410187
		background-poly_2 : 2.554959550281556
		background-A : 55811144277.94749
		background-r : 1.8223351721836667
		spectrum : energy_loss:  energy-loss (energy_loss) of size (2048,)
		blurred : [1564512.9  1567874.5  1574252.1  ...  343700.06  343373.5   343206.78]
		mask : [0. 0. 0. ... 1. 1. 1.]
		fit_parameter : [ 6.84535626e+13  3.04470742e+14  2.98252875e+05  2.98252875e+05
  2.98252875e+05  2.98252875e+05  6.30802096e+05 -2.91551529e+03
  2

## ELNES
The electron energy-loss near edge structure is determined by fititng the spectrum after quantification model subtraction. 

First smooth the spectrum (only 0 iteration available at this time) and then 
find the number of peaks you want (Can be repeated as oftern as one wants).

Please note that the peaks do not necessarily have a physcial meaning as there are peaks with 
positive and negative amplitudes. The goal is to get a smooth representation of the spectrum.
A difference of a good fit and the data should result in a residual of noise only. To achieve such a good fit, the careful setting of the edge onset is important.


In [139]:
peakFitWidget = ieels.PeakFitWidget(datasets)

AppLayout(children=(GridspecLayout(children=(Button(description='Fit Area', layout=Layout(grid_area='widget001…

### Output

In [133]:
areas = []
for p, peak in peakFitWidget.peaks['peaks'].items():
    area = np.sqrt(2* np.pi)* peak['amplitude'] * np.abs(peak['width'] / np.sqrt(2 *np.log(2))) 
    areas.append(area)
    if 'associated_edge' not in peak:
        peak['associated_edge']= ''
    print(f"peak  {p}: position: {peak['position']:7.1f}, area: {area:12.3f} associated edge: {peak['associated_edge']}")
#print(f'\n M4/M5 peak 2 to peak 1 ratio: {(areas[1])/areas[0]:.2f}')

peak  0: position:  1137.3, area:  -902696.326 associated edge: 
peak  1: position:   397.7, area:   248345.905 associated edge: 
peak  2: position:   471.7, area:   358866.925 associated edge: Ti-L2
peak  3: position:   474.5, area:  -214907.993 associated edge: Ti-L2
peak  4: position:   467.9, area:   127961.868 associated edge: Ti-L2
peak  5: position:   462.5, area:    87493.436 associated edge: Ti-L3
peak  6: position:   483.9, area:  -125417.193 associated edge: Ti-L2
peak  7: position:   470.7, area:   -88361.367 associated edge: Ti-L2
peak  8: position:   457.8, area:   -48395.811 associated edge: 
peak  9: position:   465.5, area:    46623.179 associated edge: Ti-L2
peak  10: position:   460.2, area:    38463.028 associated edge: Ti-L3


### Log Data
All the data associated with quantification are stored in the ``metadata`` attribute. 
The new key in that dictionary is: ``peak_fit``.

In [55]:
infoWidget.dataset.metadata.keys()

dict_keys(['filename', 'edges', 'experiment'])

## Save and Close File
The dataset with the metadata should be saved for further quantification. 
The file needs to be closed to be used with other notebooks.

In [105]:
h5_group = ft.save_dataset(infoWidget.datasets, '../../example_data/EELS_STO3.hf5')
h5_group.file.close()

C:\Users\gduscher\AppData\Local\anaconda3\envs\pyter\lib\site-packages\pyNSID\io\hdf_utils.py:376: FutureWarning: validate_h5_dimension may be removed in a future version
  warn('validate_h5_dimension may be removed in a future version',
C:\Users\gduscher\AppData\Local\anaconda3\envs\pyter\lib\site-packages\pyNSID\io\hdf_utils.py:376: FutureWarning: validate_h5_dimension may be removed in a future version
  warn('validate_h5_dimension may be removed in a future version',
C:\Users\gduscher\AppData\Local\anaconda3\envs\pyter\lib\site-packages\pyNSID\io\hdf_io.py:111: UserWarning: main_data_name should not contain the "-" character. Reformatted name from:collection-60muem to collection_60muem
  warn('main_data_name should not contain the "-" character. Reformatted'
C:\Users\gduscher\AppData\Local\anaconda3\envs\pyter\lib\site-packages\pyNSID\io\hdf_utils.py:376: FutureWarning: validate_h5_dimension may be removed in a future version
  warn('validate_h5_dimension may be removed in a future